In [12]:
def read_steps(file_path):
    steps = []
    with open(file_path, 'r') as file:
        lines = file.readlines()[2:]  
        for line in lines:
            parts = line.strip().split('|')
            print(parts)
            if len(parts) >= 3:
                action = parts[2].strip()
                expected_result = parts[3].strip()
                steps.append((action, expected_result))
    return steps

In [13]:
m = read_steps("steps.txt")

['', ' 1 ', ' Log in to Digital Customer Portal as Admin ', ' Landing page is displayed ', '']
['', ' 2 ', ' Click "Admin" icon on top ribbon ', ' Page with 3 options for admins is displayed ', '']
['', ' 3 ', ' Verify that page contains following tiles:- User Management<br>- Escalation Criteria<br>- Admin Ruleset ', ' Page contains tiles:- User Management<br>- Escalation Criteria<br>- Admin Ruleset ', '']
['', ' 4 ', ' Click on tab "User Management" ', ' User management page is displayed with list of all users ', '']
['', ' 5 ', ' Enter GID, part of User name or e-mail in to search field ', ' Only users matching search conditions are displayed ', '']
['', ' 6 ', ' Click "Edit" button ', ' Popup with user details, assigned roles and teams is displayed.For reference check attached screenshot ', '']
['', ' 7 ', ' Click "Upate User" button ', ' User management page is dipslayed users matching search conditions ', '']


In [21]:
def read_java_code(file_path):
    with open(file_path, 'r') as file:
        return file.read()
    
# /Users/jitender/Documents/GitHub/siemens-code-gen/finetuning/test.ipynb
def extract_code_segments(java_code_path, steps):
    code_segments = []
    # Split the Java code into lines
    java_code = read_java_code(java_code_path)
    lines = java_code.splitlines()
    # Initialize a variable to keep track of the current line index
    current_line = 0
    capturing = False  # Flag to indicate if we are capturing lines

    for action, expected in steps:
        segment = []
        while current_line < len(lines):
            line = lines[current_line].strip()
            if line.startswith("//Step"):
                if capturing:  # If we were capturing, save the segment
                    code_segments.append("\n".join(segment).strip())
                    segment = []  # Clear the segment for the next step
                capturing = True  # Start capturing after the first step
            elif capturing:  # Only capture lines if we are in a capturing state
                segment.append(line)
            current_line += 1
        
        # Append the last segment if capturing was active
        if segment:
            code_segments.append("\n".join(segment).strip())

    return code_segments

In [22]:
extract_code_segments("output.java", m)

['tc.browser.start(WebDrv.EDGE, ETestData.QA_ENV_URL, new CoreStartOptions());\ntc.browser.localLogin();\nWaitFor.condition(() -> tc.tile.exists(ETile.REPORT_AN_ISSUE));\ntc.addStepInfo("Landing page is Displayed", true, tc.button.exists(EButton.CONTACT),\nnew ComparerOptions().takeScreenShotPlatform());',
 'WaitFor.condition(() -> tc.button.exists(EButton.ADMIN));\ntc.button.click(EButton.ADMIN);\nWaitFor.condition(() -> tc.tile.exists(ETile.USER_MANAGEMENT));\nList<String> actualTiles = tc.tile.getAllTilesName();\ntc.addStepInfo("Page with 3 options for admins is displayed", 3, actualTiles.size(),\nnew ComparerOptions().takeScreenShotPlatform());',
 'List<String> expectedTiles = Arrays.asList("User Management", "Escalation Criteria" ,"Admin Ruleset");\ntc.addStepInfo("""\nPage contains tiles:\n- User Management\n- Escalation Criteria\n- Admin Ruleset""", true, actualTiles.equals(expectedTiles), new ComparerOptions().takeScreenShotPlatform());',
 'tc.tile.open(ETile.USER_MANAGEMENT);\